#Class Inheritance

Inheritance is a way to organize classes that are related in terms of the data they hold or their functionality.  Child classes inherit attributes from their parents, so we don't have to recreate the same functionality in different places.  If we have to update a particular attribute, we usually only have to do that in one place.  Inheritance also helps us think about how classes are related, since a child class will generally extend or specialize the functionality of a parent.

Let's see how inheritance works by performing some simulations.  Simulation is a great place to start using classes, because each class has a real-world counterpart, which makes it easy to think about.  For example, we previously designed our drone class by thinking about the properties of a real-world drone.  As we will see later, object-oriented programming requires us to move past such easy analogies to more abstract classes.  For now, however, we will continue to work with simulations as an easy way to build up our familiarity with classes.

In particular, let's use a class to represent a discrete-time stochastic process.  A stochastic process can be thought of as a single variable that changes over time.  It may help to think of the price of a stock over time, or measurements of solar radiation, or the average planetary surface temperature.

For every modeled time t, a stochastic process has a value, which is a real number.  A discrete process proceeds from each integer time t to time t + 1.  This process may be deterministic or probabilistic, depending on the process.  As object-oriented programmers, we might immediately think about a time_step() method to represent this process.

In [2]:
class Process:
    """Representation of a Stochastic Process"""
    def __init__(self, start_value = 0):
        self.value = start_value
        
    def time_step(self):
        pass

So far, our class is pretty empty.  The time_step function doesn't do anything.  This is what we might call an abstract base class - it defines behaviors we think a process should have, but doesn't actually fill those behaviors in.  

At times, you may see this done a different way.  We could make our time_step method raise a NotImplementedError.  This is a technique that actually forces people subclassing your class to override the method, but for this example, we'll just leave an empty method.

Let's subclass our Process with a concrete example of a process.  We're use a linear process, one that changes a fixed amount in every time period.  

To help with our visualization, we'll assume our process is bounded between 0 and 1, and we'll reverse its velocity whenever it hits a bound.

In [11]:
class BoundedLinearProcess(Process):
    """A stochastic process that develops linearly, but bounded within 0-1.
    The velocity attribute is the amount the value changes in each time period,
    and it is reset to -velocity whenever the process reaches 0 or 1."""
    def __init__(self, start_value = 0, velocity = 0):
        super().__init__(start_value)
        self.velocity = velocity
    
    def time_step(self):
        self.value += self.velocity
        if self.value < 0:
            self.value = -self.value
            self.velocity = -self.velocity
        if self.value > 1:
            self.value = 1 - (self.value - 1)
            self.velocity = -self.velocity
        super().time_step


Notice that our `__init__` method begins with a call to super().  This special function returns the parent class of the upper class.  In other words, we begin by calling the `__init__` function of `Process`.  notice that we pass in the `start_value` parameter, so that the parent initializer records this value.

Let's test the process to see if it moves as expected.

In [4]:
p1 = BoundedLinearProcess(0,.3)
print(p1)
print("current process value:",p1.value)
p1.time_step()
print("current process value:",p1.value)
p1.time_step()
print("current process value:",p1.value)
p1.time_step()
print("current process value:",p1.value)
p1.time_step()
print("current process value:",p1.value)

current process value: 0
current process value: 0.3
current process value: 0.6
current process value: 0.8999999999999999
current process value: 0.8


Before we continute, let's make a small improvement that will help us diagnose what our class is doing.  Notice that we printed out out object just to make sure we created it successfully.  When we print `p1`, however, the output isn't very nice.  It gives us the memory location of our object, but that's not something we usually care about. 

Here's a trick to make printing our class a lot nicer.  We'll add a special method, `__str__`, to return a more informative string.

In [5]:
print(p1)

In [6]:
class Process:
    def __init__(self, start_value = 0):
        self.value = start_value
        
    def time_step(self):
        pass
    
    def __str__(self):
        return "Process with current value " + str(self.value)

In [7]:
p1 = BoundedLinearProcess(0,.3)
print(p1)
p1.time_step()
print(p1)
p1.time_step()
print(p1)
p1.time_step()
print(p1)
p1.time_step()
print(p1)

There are actually two methods that do similar things: `__str__` and `__repr__`.  If one doesn't work, try the other one.  These are called magic methods, and we'll have more to say about them later.

Since we know our BoundedLinearProcess stays between 0 and 1, we can override the `__str__` method to print a basic text representation of the process.

In [36]:
class BoundedLinearProcess(Process):
    def __init__(self, start_value = 0, velocity = 0):
        super().__init__(start_value)
        self.velocity = velocity
    
    def time_step(self):
        self.value += self.velocity
        if self.value < 0:
            self.value = -self.value
            self.velocity = -self.velocity
        if self.value > 1:
            self.value = 1 - (self.value - 1)
            self.velocity = -self.velocity
        super().time_step

    def __str__(self):
        return " " * int(self.value*20) + "*"

In [10]:
p1 = BoundedLinearProcess(0,.1)
for i in range(20):
    print(p1)
    p1.time_step()

*
  *
    *
      *
        *
          *
            *
              *
               *
                  *
                   *
                  *
                *
              *
            *
          *
        *
      *
    *
  *


Let's make another process we can play around with.  An autoregressive process of order 1, also called an AR(1) process, is one in which the value at time t is given by, $x_t = \alpha x_{t-1} + w_t$ where $\alpha$ is a constant, $x_{t-1}$ is the previous value, and $w_t$ is a white noise term that is drawn from a normal distribution with standard deviation $\sigma$.

In [39]:
import numpy as np

class ARProcess(Process):
    
    def __init__(self, alpha = 0.5, sigma = 1, start_value = 0):
        super().__init__(start_value)
        self.alpha = alpha
        self.sigma = sigma
        
    def time_step(self):
        self.value = self.alpha * self.value + np.random.normal(scale = self.sigma)
        super().time_step()
        
    def __str__(self):
        if self.value<0:
            s = " " * int(5 * (self.value + 3)) + "*" + " " * int(-self.value * 5) + "|"
        elif self.value== 0:
            s = " " * 15 + "*"
        else:
            s = " " * 15 + "|" + " " * int(5 * self.value) + "*"
        return s

In [30]:
p2 = ARProcess()
for i in range(20):
    print(p2)
    p2.time_step()

               *
           *   |
               |    *
               | *
           *   |
         *     |
               |  *
          *    |
        *      |
              *|
               |        *
               |      *
               |          *
            *  |
            *  |
               |     *
               |    *
          *    |
               | *
               |  *


Let's increase our AR term, alpha, to 0.9 and see the effect on the process.  This new process should show more persistence.  A high value tends to be followed by more high values and a low value tends to be followed by more low values.

In [ ]:
p2 = ARProcess(alpha = 0.9)
for i in range(20):
    print(p2)
    p2.time_step()

As we've seen, we often want to simulate a process several times and observe the results.  Let's create a method to do this.  Since this is a common task for all Processes, we'll want to put it in the Process class.

In [38]:
class Process:
    def __init__(self, start_value = 0):
        self.value = start_value
        
    def time_step(self):
        pass
    
    def __str__(self):
        return "Process with current value " + str(self.value)
    
    def simulate(self, steps = 20):
        for i in range(steps):
            print(self)
            self.time_step()

In [40]:
p1 = BoundedLinearProcess(0,.1)
p2 = ARProcess(alpha = 0.9)

In [42]:
p1.simulate()

*
 *
   *
     *
       *
         *
           *
             *
               *
                 *
                   *
                  *
                *
              *
            *
          *
        *
      *
    *
  *


In [43]:
p2.simulate()

               *
         *     |
          *    |
             * |
        *      |
               |*
          *    |
               |  *
               | *
               |   *
               |           *
               |          *
               |    *
               |        *
               |     *
               |        *
               |   *
               |        *
               |             *
               |             *


Notice that our simulate function works with both child classes.  In one case, self refers to a BoundedLinearProcess, in the other, to an ARProcess.  This is an example of what we call duck typing. Python doesn't care what class self refers to.  All that matters is that self has a time_step() method so that the simulate function can run.

Let's make one more subclass to show how easy it is to reuse code with class inheritance.  A random walk process is a process of the form $x_t = x_{t-1} + w_t$.  Here, $w_t$ is again a white noise term drawn from a normal distribution with standard deviation $sigma$.

Statistically speaking, a random walk is an example of what we call a non-stationary process.  A statistician would say that it is not an autoregressive process, but we can implement it by creating an AR(1) process with attribute `alpha = 1`.

In [45]:
class RandomWalk(ARProcess):
    def __init__(self, sigma = .5):
        super().__init__(alpha = 1, sigma = sigma)

In [46]:
p3 = RandomWalk()
p3.simulate()

               *
               |  *
               |          *
               |           *
               |           *
               |               *
               |                *
               |                   *
               |                 *
               |              *
               |              *
               |              *
               |             *
               |            *
               |            *
               |               *
               |                  *
               |              *
               |                *
               |                *


The random walk process has no tendency to revert to any fixed mean.  It will eventually wander too far away from zero and our nice print statement won't work anymore.  Notice how easy it was to create this new class, utilizing all the machinery found in its parent classes.